In [ ]:
!pip install chess

In [ ]:
!pip install CairoSVG

In [ ]:
!pip install opencv-contrib-python

In [4]:
import chess
import chess.pgn
import io
import os

import chess.svg
from cairosvg import svg2png

import time
from IPython.display import display, HTML, clear_output

In [5]:
os.mkdir('multimedia')
os.mkdir('temp')
os.mkdir('anki')

In [6]:
def save_board_game(board, uci, cont):
  boardsvg = chess.svg.board(board=board)
  name_board = str(cont) + str(uci)
  name_file = name_board 
  f = open("./temp/" + name_file + ".svg", "w")
  f.write(boardsvg)
  f.close()

  svg_code = open("./temp/" + name_file + ".svg", "rt").read()
  svg2png(bytestring=svg_code,write_to="./multimedia/" + name_file + ".png")


def save_board_stats(board, cont):
  boardsvg = chess.svg.board(board=board)
  name_board = str(cont) + "statusboard" # NAME BOARD INIT
  name_file = name_board 
  f = open("./temp/" + name_file + ".svg", "w")
  f.write(boardsvg)
  f.close()

  svg_code = open("./temp/" + name_file + ".svg", "rt").read()
  svg2png(bytestring=svg_code,write_to="./multimedia/" + name_file + ".png")

In [ ]:
def who(player):
    return "White" if player == chess.WHITE else "Black"


def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"


def play_game(player1, player2, visual="svg", pause=0.1, cont=1):
    """
    playerN1, player2: functions that takes board, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board() # Notation Forsyth-Edwards
    save_board_stats(board, cont)
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            name = who(board.turn)
            board.push_uci(uci)
            
            cont += 1
            save_board_game(board, uci, cont)

            board_stop = display_board(board, use_svg)

            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    return (result, msg, board)


def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

def human_player(board):
    display(board)
    uci = get_move("%s's move [q to quit]> " % who(board.turn))
    legal_uci_moves = [move.uci() for move in board.legal_moves]
    while uci not in legal_uci_moves:
        print("Legal moves: " + (",".join(sorted(legal_uci_moves))))
        uci = get_move("%s's move[q to quit]> " % who(board.turn))
    return uci

In [ ]:
play_game(human_player, human_player)

In [ ]:
import os
import imageio
import shutil
import cv2

In [ ]:
# Generate files to anki

name_fl = input("Name file: >>> ")
path = './multimedia/'
archivos = sorted(os.listdir(path))

def gif(name_fl,archivos,path):
  img_array_png = []
  #Leer todos los archivos formato imagen desde path
  for x in range(0, len(archivos)):
      nomArchivo = archivos[x]
      dirArchivo = path + str(nomArchivo)
      
      #Asignar a variable leer_imagen, el nombre de cada imagen
      leer_imagen = imageio.imread(dirArchivo)
      # añadir imágenes al arreglo img_array
      img_array_png.append(leer_imagen)
  #Guardar Gif
  imageio.mimwrite("./anki/" + name_fl + ".gif", img_array_png, 'GIF', duration=1.0)
gif(name_fl,archivos,path)

def mp4(name_fl, archivos,path):
  lista_mult_mp4 = []
  for x in range(0, len(archivos)):
      nomArchivo = archivos[x]
      dirArchivo = path + str(nomArchivo)
      lista_mult_mp4.append(dirArchivo)

  img_array_mp4 = []
  for x in lista_mult_mp4:
      img = cv2.imread(x)
      img_array_mp4.append(img)

  video = cv2.VideoWriter("./anki/" + name_fl + ".mp4", cv2.VideoWriter_fourcc(*'mp4v'), 2, (390,390))

  for i in range(len(img_array_mp4)):
    video.write(img_array_mp4[i])

  video.release() 

mp4(name_fl,archivos,path)

In [ ]:
lista_multi = []
for x in range(0, len(archivos)):
    nomArchivo = archivos[x]
    dirArchivo = path + str(nomArchivo)
    lista_multi.append(dirArchivo)

board_init = lista_multi[0]
shutil.copy(board_init,"./anki/" + name_fl + ".png")

In [ ]:
# Reset
!rm -rf anki
!rm -rf multimedia
!rm -rf temp